## TRABALHO PRÁTICO 01

- O trabalho prático deverá ser feito em dupla.
- A realização da entrega deverá ser feita via Teams (tarefa adicionada à equipe), atente-se ao prazo de entrega. Não será possível realizar a entrega após o prazo previsto.
- Apenas esse arquivo (.ipynb) com a resolução deverá ser entregue (entregas em formato .zip serão penalizados)
- Apenas 1 aluno da dupla deverá fazer a entrega e colocar o nome da dupla.

#### Preencha com as informações da dupla
#### a) Antônio Victor Mendes Fonseca / GES / 7º / C210-L3
#### b) NOME SOBRENOME / CURSO / PERÍODO / TURMA

___

### 1) Problema de buscas.
Considere um tabuleiro quadrado onde cada bloco pode estar LIMPO "o" ou SUJO "i", seu trabalho é mover o aspirador "x" pelo tabuleiro a fim de limpar todos os blocos sujos. 
- Sempre que o aspirador deixa um bloco, esse pode ser considerado como limpo.
- O objetivo é deixar todos espaços limpos (tabuleiro preenchido apenas com 'o' e com o aspirador 'x')
- Para responder às questões a, b, c, d basta completar os códigos 

<img src="images/problema.png" width = 150>

- Exemplo de buscas

<img src="images/buscas.png" width = 1000>

Classe do problema (não é necessário alterá-la):

In [1]:
import numpy as np

class Aspirador():
    def __init__(self, tamanho):
        '''
        Construtor
        Args:
            - tamanho: quantidade de linhas e colunas
        '''
        self.tamanho = tamanho
        
    def encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.tamanho):
            for j in range(self.tamanho):
                if estado[i, j] == elemento:
                    return i, j
        return None, None
    
    def verifica_objetivo(self, estado):
        '''
        Verifica se o estado atual é o objetivo
        Objetivo: Não haver sujeira ("i") -> Todos blocos, exceto onde o aspirador está
        devem ser "o"
        Args:
            - estado: estado atual do tabuleiro
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        item, cont = np.unique(estado, return_counts = True)
        mapa = dict()
        for i in range(len(item)):
            mapa[item[i]] = cont[i]
            
        # Todos elementos exceto onde o aspirador está
        if mapa['o'] == (self.tamanho**2 - 1):
            return True
        
        return False
    
    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self.encontra_posicao(atual, 'x')

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        return novos_estados
        

#### a) Implementar a função de busca em largura

In [2]:
from queue import Queue

class BreadthFirstSearch():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    
    def busca(self, inicio):
        '''
        Realiza a busca BFS, armazenando os estados em uma FILA
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        fila = Queue()
        fila.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not fila.empty():
            atual = fila.get()
            estados_visitados.append(atual)
           
            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                print(f"Visitando #{cont_estados}")
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        fila.put(i)
                        
        return solucao_encontrada, estados_visitados, cont_estados

#### b) Implementar a função de busca em profundidade

In [3]:
from queue import LifoQueue

class DepthFirstSearch():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    def busca(self, inicio):
        
        piha = LifoQueue()
        piha.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not piha.empty():
            atual = piha.get()
            estados_visitados.append(atual)
            
            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                print(f"Visitando #{cont_estados}")
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        piha.put(i)

        return solucao_encontrada, estados_visitados, cont_estados

#### c) Implemente a busca gulosa com a seguinte heurística:
#### h(n) = nº de espaços com sujeira "i" (Minimizar a quantidade de estados com sujeira)

In [4]:
class Aspirador():
    def __init__(self, tamanho):
        '''
        Construtor
        Args:
            - tamanho: quantidade de linhas e colunas
        '''
        self.tamanho = tamanho
        
    def encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.tamanho):
            for j in range(self.tamanho):
                if estado[i, j] == elemento:
                    return i, j
        return None, None
    
    def verifica_objetivo(self, estado):
        '''
        Verifica se o estado atual é o objetivo
        Objetivo: Não haver sujeira ("i") -> Todos blocos, exceto onde o aspirador está
        devem ser "o"
        Args:
            - estado: estado atual do tabuleiro
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        item, cont = np.unique(estado, return_counts = True)
        mapa = dict()
        for i in range(len(item)):
            mapa[item[i]] = cont[i]
            
        # Todos elementos exceto onde o aspirador está
        if mapa['o'] == (self.tamanho**2 - 1):
            return True
        
        return False
    
    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self.encontra_posicao(atual, 'x')

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        return novos_estados
        
    def heuristica(self, estado):
        n_sujos = 0 
        
        for i in range(self.tamanho):
            for j in range(self.tamanho):
                if estado[i, j] == 'i':
                    n_sujos+=1
                    
        return n_sujos

In [5]:
import heapq

class BuscaGulosa():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    def busca(self, inicio):
        p_fila = []
        
        # H, ID, elemento
        id_estado = 0
        heapq.heappush(p_fila, (0, id_estado, inicio))

        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)
            print(f"Estado:\n{atual}")

            if self.problema.verifica_objetivo(atual):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estados(atual)
                print("Estados Gerados:")
                for i in novos_estados:
                    if not self.verifica_visitados(i, estados_visitados):
                        id_estado += 1
                        n_sujos = self.problema.heuristica(i)
                        print(i)
                        print(n_sujos)
                        heapq.heappush(p_fila, (n_sujos, id_estado, i))

        return solucao_encontrada, estados_visitados, cont_estados

#### d) Execute as 3 buscas com o tabuleiro inicial fornecido e compare: o tempo de execução de cada busca, a quantidade de estados buscados (inclusive o estado objetivo) e se a solução foi encontrada ou não.

In [6]:
# Pacote auxiliar para o cálculo do tempo
from time import time

# Criando objeto do problema
problema = Aspirador(3)

# Criando Matriz inicial
start = np.array([['x','i','o'],['i','i','i'],['o','o','i']])
start

array([['x', 'i', 'o'],
       ['i', 'i', 'i'],
       ['o', 'o', 'i']], dtype='<U1')

In [7]:
# Execução do BFS
bfs = BreadthFirstSearch(problema)

ini = time() # Tempo inicial

bfs_solucao, bfs_estados_visitados, bfs_num_visitados = bfs.busca(start) # chamando busca

bfs_time = time()-ini # Tempo total

if bfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Visitando #4
Visitando #5
Visitando #6
Visitando #7
Visitando #8
Visitando #9
Visitando #10
Visitando #11
Visitando #12
Visitando #13
Visitando #14
Visitando #15
Visitando #16
Visitando #17
Visitando #18
Visitando #19
Visitando #20
Visitando #21
Visitando #22
Visitando #23
Visitando #24
Visitando #25
Visitando #26
Visitando #27
Visitando #28
Visitando #29
Visitando #30
Visitando #31
Visitando #32
Visitando #33
Visitando #34
Visitando #35
Visitando #36
Visitando #37
Visitando #38
Visitando #39
Visitando #40
Visitando #41
Visitando #42
Visitando #43
Visitando #44
Visitando #45
Visitando #46
Visitando #47
Visitando #48
Visitando #49
Visitando #50
Visitando #51
Visitando #52
Visitando #53
Visitando #54
Visitando #55
Visitando #56
Visitando #57
Visitando #58
Visitando #59
Visitando #60
Visitando #61
Visitando #62
Visitando #63
Visitando #64
Visitando #65
Visitando #66
Visitando #67
Visitando #68
Visitando #69
Visitando #70
Visitando #71
Visitando #72
V

In [8]:
# Execução do DFS
dfs = DepthFirstSearch(problema)

ini = time() # Tempo inicial

dfs_solucao, dfs_estados_visitados, dfs_num_visitados = dfs.busca(start) # chamando busca

dfs_time = time()-ini # Tempo total

if dfs_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Visitando #1
Visitando #2
Visitando #3
Visitando #4
Visitando #5
Visitando #6
Visitando #7
Visitando #8
Visitando #9
Solution found!!!


In [9]:
# Execução do BG
bg = BuscaGulosa(problema)

ini = time() # Tempo inicial

bg_solucao, bg_estados_visitados, bg_num_visitados = bg.busca(start) # chamando busca

bg_time = time()-ini # Tempo total

if bg_solucao:
    print(f"Solution found!!!")
else:
    print("Solution not found!!!")

Estado:
[['x' 'i' 'o']
 ['i' 'i' 'i']
 ['o' 'o' 'i']]
Visitando # 1
Estados Gerados:
[['o' 'i' 'o']
 ['x' 'i' 'i']
 ['o' 'o' 'i']]
4
[['o' 'x' 'o']
 ['i' 'i' 'i']
 ['o' 'o' 'i']]
4
Estado:
[['o' 'i' 'o']
 ['x' 'i' 'i']
 ['o' 'o' 'i']]
Visitando # 2
Estados Gerados:
[['x' 'i' 'o']
 ['o' 'i' 'i']
 ['o' 'o' 'i']]
4
[['o' 'i' 'o']
 ['o' 'i' 'i']
 ['x' 'o' 'i']]
4
[['o' 'i' 'o']
 ['o' 'x' 'i']
 ['o' 'o' 'i']]
3
Estado:
[['o' 'i' 'o']
 ['o' 'x' 'i']
 ['o' 'o' 'i']]
Visitando # 3
Estados Gerados:
[['o' 'x' 'o']
 ['o' 'o' 'i']
 ['o' 'o' 'i']]
2
[['o' 'i' 'o']
 ['o' 'o' 'i']
 ['o' 'x' 'i']]
3
[['o' 'i' 'o']
 ['x' 'o' 'i']
 ['o' 'o' 'i']]
3
[['o' 'i' 'o']
 ['o' 'o' 'x']
 ['o' 'o' 'i']]
2
Estado:
[['o' 'x' 'o']
 ['o' 'o' 'i']
 ['o' 'o' 'i']]
Visitando # 4
Estados Gerados:
[['o' 'o' 'o']
 ['o' 'x' 'i']
 ['o' 'o' 'i']]
2
[['x' 'o' 'o']
 ['o' 'o' 'i']
 ['o' 'o' 'i']]
2
[['o' 'o' 'x']
 ['o' 'o' 'i']
 ['o' 'o' 'i']]
2
Estado:
[['o' 'i' 'o']
 ['o' 'o' 'x']
 ['o' 'o' 'i']]
Visitando # 5
Estados Gerados:

In [10]:
# Apresentando resultados
print("==== BFS ====")
print(f"Solução encontrada? {bfs_solucao}")
print(f"Número de estados visitados: {bfs_num_visitados}")
print(f"Tempo de execução: {bfs_time}")

print("==== DFS ====")
print(f"Solução encontrada? {dfs_solucao}")
print(f"Número de estados visitados: {dfs_num_visitados}")
print(f"Tempo de execução: {dfs_time}")

print("==== BG ====")
print(f"Solução encontrada? {bg_solucao}")
print(f"Número de estados visitados: {bg_num_visitados}")
print(f"Tempo de execução: {bg_time}")

==== BFS ====
Solução encontrada? True
Número de estados visitados: 89
Tempo de execução: 0.042205810546875
==== DFS ====
Solução encontrada? True
Número de estados visitados: 9
Tempo de execução: 0.006646156311035156
==== BG ====
Solução encontrada? True
Número de estados visitados: 9
Tempo de execução: 0.008975982666015625
